<a href="https://colab.research.google.com/github/tashisharma10/credit-card-fraud-detection-/blob/main/Credit_Card_fraud_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [344]:
#loading the dataset into pandas Dataframe
import csv
filename = '/content/creditcard.csv'


def read_csv_with_error_handling(filename):
    data = []
    with open(filename, 'r', newline='', encoding='utf-8') as file:
        reader = csv.reader(file)
        header = next(reader)
        data.append(header)
        for line_number, row in enumerate(reader, start=2):
            try:
                if len(row) != len(header):
                    raise ValueError(f"Unexpected number of fields in line {line_number}")
                data.append(row)
            except ValueError as e:
                print(f"Skipping line {line_number}: {e}")

    # Convert data list to DataFrame
    df = pd.DataFrame(data[1:], columns=data[0])
    return df

try:
    dataset = read_csv_with_error_handling(filename)
    print("CSV file loaded successfully.")
except Exception as e:
    print(f"Error loading CSV file: {e}")

CSV file loaded successfully.


In [289]:
dataset.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,class
0,0,-1.359807134,-0.072781173,2.536346738,1.378155224,-0.33832077,0.462387778,0.239598554,0.098697901,0.36378697,...,-0.018306778,0.277837576,-0.11047391,0.066928075,0.128539358,-0.189114844,0.133558377,-0.021053053,149.62,0
1,0,1.191857111,0.266150712,0.166480113,0.448154078,0.060017649,-0.082360809,-0.078802983,0.085101655,-0.255425128,...,-0.225775248,-0.638671953,0.101288021,-0.339846476,0.167170404,0.125894532,-0.008983099,0.014724169,2.69,0
2,1,-1.358354062,-1.340163075,1.773209343,0.379779593,-0.503198133,1.800499381,0.791460956,0.247675787,-1.514654323,...,0.247998153,0.771679402,0.909412262,-0.689280956,-0.327641834,-0.139096572,-0.055352794,-0.059751841,378.66,0
3,1,-0.966271712,-0.185226008,1.79299334,-0.863291275,-0.01030888,1.247203168,0.23760894,0.377435875,-1.387024063,...,-0.108300452,0.005273597,-0.190320519,-1.175575332,0.647376035,-0.221928844,0.062722849,0.061457629,123.50,0
4,2,-1.158233093,0.877736755,1.548717847,0.403033934,-0.407193377,0.095921462,0.592940745,-0.270532677,0.817739308,...,-0.009430697,0.798278495,-0.13745808,0.141266984,-0.206009588,0.502292224,0.21942223,0.215153147,69.99,0


In [ ]:
dataset.tail()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
239674,75998,-1.53451294273687,1.3345229933693,0.460652595005597,-1.463070846436,-0.333040202864263,-0.343935007711465,-0.175898619512967,1.01122108715847,-0.472495264083057,...,0.116695393069048,-0.0122338338543161,-0.00272037890230054,-0.275184855806462,-0.344329583374037,0.776453984075253,-0.264331752528659,0.0288131499323838,0.77,0
239675,75998,1.50184494475134,-1.19314717002022,0.0667886408182965,-1.6008466932024,-1.25986485736865,-0.268558463213973,-1.12220499932983,0.0401035281584056,-1.79505991897369,...,-0.22643034356432,-0.409194359987298,-0.0742627336504342,-0.565449274488867,0.404350087691244,-0.168444593496908,0.0109806710302582,0.00162623337208679,30,0
239676,75999,-0.340454876166892,1.20175569890191,1.31409401271323,0.0531340621893484,0.111616490324281,-0.959890557470006,0.780107027682496,-0.128473160843956,-0.541651972599866,...,-0.252457948152444,-0.609958871227369,-0.0255520683422293,0.360654427033134,-0.127871442273221,0.0686012138598037,0.251848431545455,0.100380451769304,3.67,0
239677,75999,-1.08916136314755,0.115364056625668,1.50197800748259,0.700866382466051,0.493027130525983,-0.220268237377107,0.744467554316475,0.128396268747899,-0.983044080196868,...,0.391618925375883,0.776284705655453,0.179390050842909,0.218711562007416,-0.00938099334502251,-0.353212890522996,0.0824775850019173,0.143269547869868,130.8,0
239678,76000,0.213728886498821,0.62313565650925,-0.259558058640646,-0.152126985209882,2.26208811819354,3.57971164734521,-0.187220393936421,0.877580378192651,-0.697998235361168,...,-0.193588404864573,-0.663298773827501,0.0886749672797734,1.00532555031502,-0.6764117987035,0.25166074710035,0.105657956710504,0.088634421831781,1.29,0


In [ ]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 239679 entries, 0 to 239678
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   Time    239679 non-null  object
 1   V1      239679 non-null  object
 2   V2      239679 non-null  object
 3   V3      239679 non-null  object
 4   V4      239679 non-null  object
 5   V5      239679 non-null  object
 6   V6      239679 non-null  object
 7   V7      239679 non-null  object
 8   V8      239679 non-null  object
 9   V9      239679 non-null  object
 10  V10     239679 non-null  object
 11  V11     239679 non-null  object
 12  V12     239679 non-null  object
 13  V13     239679 non-null  object
 14  V14     239679 non-null  object
 15  V15     239679 non-null  object
 16  V16     239679 non-null  object
 17  V17     239679 non-null  object
 18  V18     239679 non-null  object
 19  V19     239679 non-null  object
 20  V20     239679 non-null  object
 21  V21     239679 non-null  object
 

In [ ]:
#checking the number of missing values in each column
dataset.isnull().sum()

Time      0
V1        0
V2        0
V3        0
V4        0
V5        0
V6        0
V7        0
V8        0
V9        0
V10       0
V11       0
V12       0
V13       0
V14       0
V15       0
V16       0
V17       0
V18       0
V19       0
V20       0
V21       0
V22       0
V23       0
V24       0
V25       0
V26       0
V27       0
V28       0
Amount    0
Class     0
dtype: int64

In [290]:
#distribution of legit transaction and fraudulent transaction
dataset["class"].value_counts()

class
0    284315
1       492
Name: count, dtype: int64

This data is highly imbalanced

0-Normal transaction

1-fraudulent transaction

In [291]:
print(dataset['class'].unique())

['0' '1']


In [292]:
print(dataset['class'].dtype)


object


In [293]:
dataset['class'] = dataset['class'].astype(str).str.strip()



In [297]:
# separating the data for analysis
legit = dataset[dataset['class'] == '0'].copy()
fraud = dataset[dataset['class'] == '1'].copy()

In [298]:
print("Shape of legitimate transactions data:", legit.shape)
print("Shape of fraudulent transactions data:", fraud.shape)

Shape of legitimate transactions data: (284315, 31)
Shape of fraudulent transactions data: (492, 31)


In [299]:
# Print data types of all columns
print(dataset.dtypes)

Time       object
V1         object
V2         object
V3         object
V4         object
V5         object
V6         object
V7         object
V8         object
V9         object
V10        object
V11        object
V12        object
V13        object
V14        object
V15        object
V16        object
V17        object
V18        object
V19        object
V20        object
V21        object
V22        object
V23        object
V24        object
V25        object
V26        object
V27        object
V28        object
Amount    float64
class      object
dtype: object


In [303]:
#statistical measures of data
legit.Amount.describe()


count    284315.000000
mean         88.291022
std         250.105092
min           0.000000
25%           5.650000
50%          22.000000
75%          77.050000
max       25691.160000
Name: Amount, dtype: float64

In [301]:
fraud.Amount.describe()

count     492.000000
mean      122.211321
std       256.683288
min         0.000000
25%         1.000000
50%         9.250000
75%       105.890000
max      2125.870000
Name: Amount, dtype: float64

In [308]:


# Convert 'Amount' column to string type explicitly
dataset['Amount'] = dataset['Amount'].astype(str)

# Clean 'Amount' column (remove all non-numeric characters except decimal point)
dataset['Amount'] = dataset['Amount'].str.replace('[^\d.]', '', regex=True)

# Convert 'Amount' column to numeric type with errors='coerce'
dataset['Amount'] = pd.to_numeric(dataset['Amount'], errors='coerce')

# Check for NaN values after conversion
nan_values = dataset['Amount'].isna().sum()
print(f"Number of NaN values after conversion: {nan_values}")

# Drop rows with NaN values in 'Amount' column (if necessary)
dataset = dataset.dropna(subset=['Amount'])

# Verify conversion by printing unique values in 'Amount' column
print(dataset['Amount'].unique())

# Group by 'Class' and calculate mean of 'Amount' column
try:
    mean_by_class = dataset.groupby('class')['Amount'].mean()
    print(mean_by_class)
except Exception as e:
    print(f"Error calculating mean: {e}")


Number of NaN values after conversion: 0
[149.62   2.69 378.66 ... 381.05 337.54  95.63]
class
0     88.291022
1    122.211321
Name: Amount, dtype: float64


Under-Sampling

Build a sample dataset containing similar distribution of normal transactions and Fraudulent Transactions

Number of Fraudulent Transactions - 492

In [309]:
legit_sample = legit.sample(n=492)

Concatenating two DataFrames

In [310]:
new_dataset = pd.concat([legit_sample, fraud], axis=0)

In [311]:
new_dataset.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,class
19136,30042,-0.864009696,0.183136223,2.694398076,0.091609097,-0.044349033,1.848306692,-0.294303231,0.346700167,1.49205688,...,-0.135001694,0.164941726,-0.506402622,-1.261277734,0.370124574,0.582749266,-0.331877409,-0.150733414,12.63,0
218901,141519,-0.233915545,0.058800986,0.807300645,-1.629489881,-0.123415674,-0.679215324,0.648790786,-0.391061277,-0.92025572,...,0.327185024,1.042642454,-0.127904362,-0.183970581,-0.183279702,-0.149201941,-0.011386828,-0.022113149,99.00,0
68814,53154,0.035377222,0.746276397,0.764141101,1.395622418,-0.204618881,1.234426942,-0.24545333,-1.500741078,0.163650484,...,1.450810971,-0.403028626,-0.14940676,-0.729500847,1.055061322,0.006637809,0.292215403,0.203766838,72.80,0
228468,145523,1.750400419,-0.718804552,-1.827457257,0.090899423,0.922753783,1.483057506,-0.172432008,0.430715891,0.891126747,...,0.069651725,0.252331856,0.060257609,-0.957096692,-0.026629547,-0.21903054,0.005370789,-0.064344069,97.35,0
210300,137920,2.010755558,0.152902714,-1.595034043,0.329575725,0.47923019,-0.557103082,0.118878912,-0.114253989,0.116140003,...,-0.287251233,-0.719359399,0.347676332,0.629153547,-0.303847827,0.145921611,-0.058642125,-0.037861319,1.98,0


In [312]:
new_dataset.tail()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,class
279863,169142,-1.927883321,1.125652664,-4.518330641,1.749292533,-1.566487292,-2.010494231,-0.882849831,0.697211109,-2.064945287,...,0.778583979,-0.319188819,0.639418961,-0.29488504,0.537502536,0.788395057,0.292679966,0.147967929,390.00,1
280143,169347,1.378558997,1.289380937,-5.004246784,1.411849842,0.442580636,-1.326535934,-1.413169956,0.248525468,-1.127395934,...,0.370611857,0.028234445,-0.145640429,-0.081049407,0.521874502,0.739467256,0.389151844,0.186636548,0.76,1
280149,169351,-0.676142671,1.126366062,-2.213699523,0.468308388,-1.120541044,-0.003346296,-2.234739296,1.210157964,-0.65224992,...,0.751825538,0.83410769,0.190943873,0.032070086,-0.739694823,0.471110963,0.385107449,0.194361479,77.89,1
281144,169966,-3.113831607,0.585864172,-5.399730211,1.817092473,-0.840618466,-2.943547791,-2.20800192,1.058732677,-1.63233335,...,0.583275999,-0.269208638,-0.456107773,-0.18365913,-0.328167759,0.60611581,0.88487554,-0.253700319,245.00,1
281674,170348,1.991976096,0.158475887,-2.583440645,0.408669993,1.151147061,-0.096694744,0.223050267,-0.068383878,0.577829384,...,-0.164350328,-0.295135167,-0.072172531,-0.450261313,0.313266609,-0.289616586,0.002987582,-0.015308813,42.53,1


In [314]:
new_dataset['class'].value_counts()

class
0    492
1    492
Name: count, dtype: int64

In [316]:
# Convert 'Amount' column to string type explicitly
new_dataset['Amount'] = new_dataset['Amount'].astype(str)

# Clean 'Amount' column (remove all non-numeric characters except decimal point)
new_dataset['Amount'] = new_dataset['Amount'].str.replace('[^\d.]', '', regex=True)

# Convert 'Amount' column to numeric type with errors='coerce'
new_dataset['Amount'] = pd.to_numeric(new_dataset['Amount'], errors='coerce')

# Check for NaN values after conversion
nan_values = new_dataset['Amount'].isna().sum()
print(f"Number of NaN values after conversion: {nan_values}")

# Drop rows with NaN values in 'Amount' column (if necessary)
new_dataset = new_dataset.dropna(subset=['Amount'])

# Verify conversion by printing unique values in 'Amount' column
print(new_dataset['Amount'].unique())

# Group by 'Class' and calculate mean of 'Amount' column
try:
    mean_by_class = new_dataset.groupby('class')['Amount'].mean()
    print(mean_by_class)
except Exception as e:
    print(f"Error calculating mean: {e}")


Number of NaN values after conversion: 0
[1.26300e+01 9.90000e+01 7.28000e+01 9.73500e+01 1.98000e+00 4.17200e+01
 4.57000e+00 1.50000e+01 1.72000e+00 1.08950e+02 2.71280e+02 2.00300e+01
 6.64960e+02 8.99900e+01 8.99000e+00 5.61300e+01 9.71850e+02 2.30800e+01
 3.02200e+01 3.59000e+00 1.00000e+00 1.50000e+00 4.46100e+01 7.70500e+01
 2.49900e+01 5.00600e+01 7.00000e+01 1.29000e+00 2.20000e+01 9.90000e+00
 7.18000e+01 2.31000e+00 7.70000e-01 1.25000e+01 0.00000e+00 8.02000e+00
 3.14200e+01 2.87500e+01 6.40000e+01 1.56600e+01 2.54100e+01 4.91900e+01
 2.28000e+02 6.15900e+01 3.84200e+01 1.30910e+02 5.49000e+00 4.78500e+01
 7.78250e+02 3.80100e+01 1.78500e+01 9.00000e+01 1.05800e+01 3.78000e+00
 3.69000e+00 9.99000e+00 1.29900e+01 1.10230e+02 3.00000e+02 3.83400e+01
 1.53000e+00 1.96290e+02 7.82000e+01 4.91500e+01 2.49000e+00 1.96800e+01
 3.15140e+02 2.89000e+02 3.14800e+01 1.07290e+02 7.60000e-01 1.64370e+02
 2.37400e+01 1.58000e+02 9.98900e+01 4.69230e+02 1.11800e+03 4.12200e+01
 1.05200e+

Splitting the data into Features & Targets

In [317]:
X = new_dataset.drop(columns='class', axis=1)
Y = new_dataset['class']

In [318]:
print(X)

          Time            V1            V2            V3            V4  \
19136    30042  -0.864009696   0.183136223   2.694398076   0.091609097   
218901  141519  -0.233915545   0.058800986   0.807300645  -1.629489881   
68814    53154   0.035377222   0.746276397   0.764141101   1.395622418   
228468  145523   1.750400419  -0.718804552  -1.827457257   0.090899423   
210300  137920   2.010755558   0.152902714  -1.595034043   0.329575725   
...        ...           ...           ...           ...           ...   
279863  169142  -1.927883321   1.125652664  -4.518330641   1.749292533   
280143  169347   1.378558997   1.289380937  -5.004246784   1.411849842   
280149  169351  -0.676142671   1.126366062  -2.213699523   0.468308388   
281144  169966  -3.113831607   0.585864172  -5.399730211   1.817092473   
281674  170348   1.991976096   0.158475887  -2.583440645   0.408669993   

                  V5            V6            V7            V8            V9  \
19136   -0.044349033   1.848306

In [319]:
print(Y)

19136     0
218901    0
68814     0
228468    0
210300    0
         ..
279863    1
280143    1
280149    1
281144    1
281674    1
Name: class, Length: 984, dtype: object


In [337]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, stratify=Y, random_state=1)

In [338]:
print(X.shape, X_train.shape, X_test.shape)

(984, 30) (787, 30) (197, 30)


Model Training

Logistic Regression

In [339]:
model = LogisticRegression()
#training the Logistic Regression model with training data
model.fit(X_train, Y_train)
print("Logistic Regression Model Parameters:")

print(f"Intercept: {model.intercept_[0]}")
print(f"C: {model.C}")
print(f"Class weight: {model.class_weight}")
print(f"Dual: {model.dual}")
print(f"Fit intercept: {model.fit_intercept}")
print(f"Intercept scaling: {model.intercept_scaling}")
print(f"L1 ratio: {model.l1_ratio}")
print(f"Maximum iterations: {model.max_iter}")
print(f"Multi class: {model.multi_class}")
print(f"N jobs: {model.n_jobs}")
print(f"Penalty: {model.penalty}")
print(f"Random state: {model.random_state}")
print(f"Solver: {model.solver}")
print(f"Tolerance: {model.tol}")
print(f"Verbose: {model.verbose}")
print(f"Warm start: {model.warm_start}")

Logistic Regression Model Parameters:
Intercept: -0.08210614037162052
C: 1.0
Class weight: None
Dual: False
Fit intercept: True
Intercept scaling: 1
L1 ratio: None
Maximum iterations: 100
Multi class: auto
N jobs: None
Penalty: l2
Random state: None
Solver: lbfgs
Tolerance: 0.0001
Verbose: 0
Warm start: False


Model Evaluation

Accuracy Score

In [340]:
# accuracy on training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)

In [343]:
print('Accuracy on Training data : ', training_data_accuracy)

Accuracy on Training data :  0.9428208386277002


In [341]:
# accuracy on test data
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)

In [342]:
print('Accuracy score on Test Data : ', test_data_accuracy)


Accuracy score on Test Data :  0.9086294416243654
